In [12]:
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces == ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face
    
cap = cv2.VideoCapture(0)
count = 0

# Collect 200 samples of your face from webcam input
while True:

    ret, photo = cap.read()
    if face_extractor(photo) is not None:
        count += 1
        face = cv2.resize(face_extractor(photo), (200, 200)) #getting the face and changing size of image
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY) #convert colour image to black-white (grayscale)

        # Save file in specified directory with unique name
        file_name_path = r"C:\Users\Prashanth M\ML_AL\Face_Recongnition_Security_in_Linux\prashanthDataSet\\D" + str(count) + ".jpg"
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        pass

    if cv2.waitKey(1) == 27 or count == 200: #27 is the Esc Key
        break
        
cap.release()
cv2.destroyAllWindows() #upon completion close the window      
print("Collecting Samples Complete")


<ipython-input-12-d9bb7d414b42>:13: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if faces == ():


In [13]:
#To train model 

import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
print(cv2.__version__)
# Get the training dataset we Collected
data_path = r'C:\Users\Prashanth M\ML_AL\Face_Recongnition_Security_in_Linux\prashanthDataSet\\'

onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)
# 
# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)
# model=cv2.face_LBPHFaceRecognizer.create()

model = cv2.face.LBPHFaceRecognizer_create() 
# Initialize facial recognizer
#model = cv2.face_LBPHFaceRecognizer.create()
# model=cv2.f
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()

# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained successfully")



4.7.0
Model trained successfully


In [14]:
model.save("Prashanth_M.model")

In [17]:
#prediciton


import cv2
import numpy as np
import os
import urllib.request

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):

    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()

    image, face = face_detector(frame)

    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = model.predict(face)
        print(results)
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'

        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)

        if confidence > 85:
           
            cv2.putText(image, "Welcome Prashanth!!", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    
            
            #API Connection to AWS to unlock
            request_url = urllib.request.urlopen("https://5kmrmnmj12.execute-api.ap-south-1.amazonaws.com/ai_ml/unlock")
            print( request_url.read())
            cv2.imshow('Face Recognition', image )
            break;
            

        else:
            cv2.putText(image, "i dont know", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            #API Connection to AWS to lock
            request_url = urllib.request.urlopen("https://5kmrmnmj12.execute-api.ap-south-1.amazonaws.com/ai_ml/lock")
            print( request_url.read())
            cv2.imshow('Face Recognition', image )
           

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass

    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()


<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-17-2c47c035bcc4>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


(1, 68.49898451623014)
b'{"statusCode": 200, "body": "\\"User Locked!\\""}'
(162, 93.78054451921716)
b'{"statusCode": 200, "body": "\\"User Locked!\\""}'
(154, 75.8296641716911)
b'{"statusCode": 200, "body": "\\"User Locked!\\""}'
(132, 35.17564329074777)
b'{"statusCode": 200, "body": "\\"User UnLocked!\\""}'
